In [1]:
import imapclient

imapclientOBJ = imapclient.IMAPClient('imap.gmail.com', ssl=True)


In [2]:
import getpass
def login():
    email = getpass.getpass('Enter Your Email: ')
    password = getpass.getpass('Enter Your secret Password: ')
    imapclientOBJ.login(email, password)
    print('You are sucessfully logged in')

In [4]:
login()

Enter Your Email:  ········
Enter Your secret Password:  ········


You are sucessfully logged in


In [5]:
imapclientOBJ.select_folder('INBOX', readonly=True)
UIDs = imapclientOBJ.search(['SINCE', '03-Jul-2024'])
UIDs[0]

3757

In [6]:
raw_messages = imapclientOBJ.fetch(UIDs[0], ['BODY[]', 'FLAGS'] )

In [ ]:
raw_messages